In [1]:
import sqlite3
import json

DB_FILE = "chunks.db"
OUTPUT_FILE = "exported_chunks.jsonl"

# Connect to the database
conn = sqlite3.connect(DB_FILE)
cur = conn.cursor()

# Query all data from chunks table
cur.execute("SELECT text, label FROM chunks")
rows = cur.fetchall()

# Write to JSONL
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for text, label in rows:
        obj = {"text": text}
        if label is not None:
            obj["label"] = label
        f.write(json.dumps(obj) + "\n")

conn.close()

print("Data exported to JSONL file.")


Data exported to JSONL file.


In [2]:
import json
import pandas as pd

# Load the labeled chunks
with open("exported_chunks.jsonl", "r", encoding="utf-8") as f:
    labeled_chunks = [json.loads(line) for line in f]

data = pd.DataFrame(labeled_chunks)
labeled_count = data['label'].value_counts().to_dict()

# Get the first 9000 rows
data = data.head(9000)

# Remove rows with label == 11
data = data[data['label'] != 11]

# Print labeled count after removing label 11
labeled_count = data['label'].value_counts().to_dict()
print(f"Labeled chunks after removing label 11: {labeled_count}")

# Remove rows where label == 1 and text length < 100
data = data[~((data['label'] == 1) & (data['text'].str.len() < 100))]

# Print final labeled count
labeled_count = data['label'].value_counts().to_dict()
print(f"Final labeled chunks: {labeled_count}")


Labeled chunks after removing label 11: {1: 8199, 0: 800}
Final labeled chunks: {1: 5384, 0: 800}


In [3]:
import pandas as pd
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)
from torch.nn import CrossEntropyLoss

# Cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading the data
data['label'] = data['label'].astype(int)

# Train-Test Split using stratified sampling
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

# since there is a class imbalance, we will compute class weights
# to handle this in the loss function
labels = train_df["label"].values
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print("Class weights:", class_weights)

# Convert ing the DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(input_data):
    return tokenizer(input_data["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Loaading the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

# Creating a custom Trainer to handle weighted loss
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
)

# Creating the Trainer
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Training the model
trainer.train()

# Evaluating the model
preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_dataset["labels"], pred_labels, digits=4))


Class weights: tensor([3.8648, 0.5743], device='cuda:0')


c:\Users\ctngweru\AppData\Local\anaconda3\envs\llm-forge-Copy\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/4947 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.

...

{'eval_loss': 0.2693607807159424, 'eval_runtime': 4.2629, 'eval_samples_per_second': 290.175, 'eval_steps_per_second': 18.297, 'epoch': 2.0}
{'loss': 0.0095, 'learning_rate': 1.6129032258064517e-05, 'epoch': 2.03}
{'loss': 0.0047, 'learning_rate': 1.5591397849462366e-05, 'epoch': 2.06}
{'loss': 0.0013, 'learning_rate': 1.5053763440860215e-05, 'epoch': 2.1}
{'loss': 0.0023, 'learning_rate': 1.4516129032258066e-05, 'epoch': 2.13}
{'loss': 0.0457, 'learning_rate': 1.3978494623655914e-05, 'epoch': 2.16}
{'loss': 0.1093, 'learning_rate': 1.3440860215053763e-05, 'epoch': 2.19}
{'loss': 0.1766, 'learning_rate': 1.2903225806451613e-05, 'epoch': 2.23}
{'loss': 0.068, 'learning_rate': 1.2365591397849464e-05, 'epoch': 2.26}
{'loss': 0.0058, 'learning_rate': 1.1827956989247313e-05, 'epoch': 2.29}
{'loss': 0.0286, 'learning_rate': 1.129032258064516e-05, 'epoch': 2.32}
{'loss': 0.0328, 'learning_rate': 1.0752688172043012e-05, 'epoch': 2.35}
{'loss': 0.3195, 'learning_rate': 1.0215053763440861e-05, '

  0%|          | 0/78 [00:00<?, ?it/s]

{'eval_loss': 0.18768875300884247, 'eval_runtime': 4.3068, 'eval_samples_per_second': 287.22, 'eval_steps_per_second': 18.111, 'epoch': 3.0}
{'train_runtime': 175.6969, 'train_samples_per_second': 84.469, 'train_steps_per_second': 5.293, 'train_loss': 0.16230862470123397, 'epoch': 3.0}


  0%|          | 0/78 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0     0.9250    0.9250    0.9250       160
           1     0.9889    0.9889    0.9889      1077

    accuracy                         0.9806      1237
   macro avg     0.9569    0.9569    0.9569      1237
weighted avg     0.9806    0.9806    0.9806      1237



In [4]:
# Running test on 10 random samples
sample_df = test_df.sample(10, random_state=42).reset_index(drop=True)

# Tokenize the samples
encodings = tokenizer(
    sample_df["text"].tolist(),
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt"
).to(device)

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**encodings)
    probs = torch.softmax(outputs.logits, dim=1)
    predictions = torch.argmax(probs, dim=1).cpu().numpy()

# Display actual vs predicted
for i in range(10):
    print(f"\n Text {i+1}:")
    print(f"Text: {sample_df.loc[i, 'text'][:300]}{'...' if len(sample_df.loc[i, 'text']) > 300 else ''}")
    print(f"Actual label   : {sample_df.loc[i, 'label']}")
    print(f"Predicted label: {predictions[i]}")



 Text 1:
Text: The federal government alerts its contractors to CI threats
and subjects them to "awareness programs" under the
DOD's Defense Information Counter Espionage (DICE)
program. The Defense Investigative Service (DIS)
maintains a host of useful databases such as the Foreign
Ownership, Control, or Influenc...
Actual label   : 1
Predicted label: 1

 Text 2:
Text: 9. The prosperity of nations rests very largely on the
      six inches of soil between the surface and the subsoil
      of the territory.
Actual label   : 1
Predicted label: 1

 Text 3:
Text: _Accept_ means _to receive_. _Except_ as a verb means _to exclude_; as a
preposition it means _with the exception of_. Insert the correct form in
the following:
Actual label   : 1
Predicted label: 1

 Text 4:
Text: How the West lost the East. The economics, the politics, the geopolitics, the
conspiracies, the corruption, the old and the new, the plough and the internet - it is all
here, in colourful and provocative prose.
Actual

In [5]:
# Testing the models generalization on new unseen data

# Random new text samples
new_texts = [
    "Photosynthesis is the process by which green plants convert sunlight into energy.",
    "Bibliography\nChapter 3\nIndex\nAcknowledgments",
    "The mitochondria is often called the powerhouse of the cell.",
    "References\n[1] Smith et al. 2022",
    "This text explains Newton’s laws of motion in detail.",
    "Appendix A: Glossary of Terms",
    "How does the water cycle work in nature?",
    "The syllabus is subject to change without notice.",
    "Cell division is crucial for reproduction in organisms.",
    "Table of contents\n1. Preface\n2. Introduction"
]

# Tokenize the new texts
encodings = tokenizer(
    new_texts,
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt"
).to(device)

# RUn the model on new texts
model.eval()
with torch.no_grad():
    outputs = model(**encodings)
    probs = torch.softmax(outputs.logits, dim=1)
    predictions = torch.argmax(probs, dim=1).cpu().numpy()

# Results 
for i, text in enumerate(new_texts):
    print(f"\nText {i+1}:")
    print(f"Text: {text[:300]}{'...' if len(text) > 300 else ''}")
    print(f"Predicted label: {predictions[i]} (1 = Relevant, 0 = Irrelevant)")



Text 1:
Text: Photosynthesis is the process by which green plants convert sunlight into energy.
Predicted label: 1 (1 = Relevant, 0 = Irrelevant)

Text 2:
Text: Bibliography
Chapter 3
Index
Acknowledgments
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 3:
Text: The mitochondria is often called the powerhouse of the cell.
Predicted label: 1 (1 = Relevant, 0 = Irrelevant)

Text 4:
Text: References
[1] Smith et al. 2022
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 5:
Text: This text explains Newton’s laws of motion in detail.
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 6:
Text: Appendix A: Glossary of Terms
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 7:
Text: How does the water cycle work in nature?
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 8:
Text: The syllabus is subject to change without notice.
Predicted label: 1 (1 = Relevant, 0 = Irrelevant)

Text 9:
Text: Cell division is crucial for reproduction in organisms.
Predicted lab

In [6]:
#save the model
model_save_path = "classifier_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)  

# using the saved model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained(model_save_path)
model = DistilBertForSequenceClassification.from_pretrained(model_save_path).to(device)

# Testing the saved model on long texts
long_texts = [

    "In this chapter, we explore the foundational principles of quantum mechanics, including the concept of wave-particle duality, uncertainty, and quantum entanglement. These phenomena have been experimentally verified and form the basis for quantum computing and quantum encryption. Students will learn how to apply Schrödinger’s equation and analyze quantum states in one-dimensional potential wells, along with real-world implications.",
    
    "Table of Contents\nPreface\nChapter 1: Introduction to Biology\nChapter 2: The Cell\nChapter 3: Genetics\nChapter 4: Evolution\nChapter 5: Ecology\nChapter 6: Human Anatomy\nChapter 7: The Immune System\nGlossary\nIndex\nAcknowledgements\nAuthor Biography\nDisclaimer: This textbook is provided as-is without warranty.",
    

    "Photosynthesis is a biochemical process that converts light energy into chemical energy in plants, algae, and some bacteria. The process involves two major stages: the light-dependent reactions and the Calvin cycle. In the light-dependent reactions, sunlight is absorbed by chlorophyll and used to split water molecules, releasing oxygen and generating ATP and NADPH. The Calvin cycle then uses these energy molecules to fix carbon dioxide into glucose, which serves as an energy source for the plant.",
    
    "Acknowledgements\nThis work would not have been possible without the support of our research assistants, editorial team, and the generous funding provided by the National Science Foundation and other partners. We also thank the numerous reviewers and contributors who helped shape the final version of this manuscript. All errors, however, remain the responsibility of the authors."
]
# Tokenize the texts
encodings = tokenizer(
    long_texts,
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt"
).to(device)
# Predicting usijng the saved model
model.eval()
with torch.no_grad():
    outputs = model(**encodings)
    probs = torch.softmax(outputs.logits, dim=1)
    predictions = torch.argmax(probs, dim=1).cpu().numpy()

for i, text in enumerate(long_texts):
    print(f"\nText {i+1}:")
    print(f"Text: {text[:300]}{'...' if len(text) > 300 else ''}")
    print(f"Predicted label: {predictions[i]} (1 = Relevant, 0 = Irrelevant)")





Text 1:
Text: In this chapter, we explore the foundational principles of quantum mechanics, including the concept of wave-particle duality, uncertainty, and quantum entanglement. These phenomena have been experimentally verified and form the basis for quantum computing and quantum encryption. Students will learn ...
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 2:
Text: Table of Contents
Preface
Chapter 1: Introduction to Biology
Chapter 2: The Cell
Chapter 3: Genetics
Chapter 4: Evolution
Chapter 5: Ecology
Chapter 6: Human Anatomy
Chapter 7: The Immune System
Glossary
Index
Acknowledgements
Author Biography
Disclaimer: This textbook is provided as-is without warr...
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 3:
Text: Photosynthesis is a biochemical process that converts light energy into chemical energy in plants, algae, and some bacteria. The process involves two major stages: the light-dependent reactions and the Calvin cycle. In the light-dependent reaction

In [7]:

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data function
def load_data(chunk_size):
    path = f"../Yaman/Generate_Paragraphs/Results/extracted_chunks_{chunk_size}_overlap.json"
    if not os.path.exists(path):
        print(f"Missing input file: {path}, skipping.")
        return None
    with open(path, "r", encoding="utf-8") as file:
        return json.load(file)

# Load and flatten data
data = load_data(256)
if data is None:
    print("No data loaded, exiting.")
    exit()

texts = []
for chunks in data.values():
    texts.extend(chunks)

# Load model
model_save_path = "classifier_model"
tokenizer = DistilBertTokenizer.from_pretrained(model_save_path)
model = DistilBertForSequenceClassification.from_pretrained(model_save_path).to(device)
model.eval()

# Inference with batching
batch_size = 16
predictions = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    encodings = tokenizer(
        batch_texts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    input_ids = encodings["input_ids"].to(device)
    attention_mask = encodings["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1).cpu().numpy()
        predictions.extend(preds)

# Display predictions
for i, text in enumerate(texts):
    print(f"\nText {i+1}:")
    print(f"Text: {text[:300]}{'...' if len(text) > 300 else ''}")
    print(f"Predicted label: {predictions[i]} (1 = Relevant, 0 = Irrelevant)")

save_path = "results/test_labeled_chunks_articles.jsonl"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, "w", encoding="utf-8") as f:
    for text, label in zip(texts, predictions):
        obj = {"text": text, "label": int(label)}
        f.write(json.dumps(obj) + "\n")





Text 1:
Text: Saylor URL: http://www.saylor.org/books 
 
Saylor.org 
 
 
1 
This text was adapted by The Saylor Foundation under a Creative 
Commons Attribution-NonCommercial-ShareAlike 3.0 License without 
attribution as requested by the work’s original creator or licensee. 
 
Saylor URL: http://www.saylor.org/b...
Predicted label: 0 (1 = Relevant, 0 = Irrelevant)

Text 2:
Text: acumen necessary for businesspeople to talk in an educated way to their customers, employees, 
suppliers, government officials—and to their own lawyers. 
 
Traditional publishers often create confusion among customers in the text selection process by 
offering a huge array of publications. Once a te...
Predicted label: 1 (1 = Relevant, 0 = Irrelevant)

Text 3:
Text: 1. Distinguish different philosophies of law—schools of legal thought—and explain their relevance. 
2. Identify the various aims that a functioning legal system can serve. 
3. Explain how politics and law are related. 
4. Identify the sources of l

In [8]:
# uploading model to huggingface hub
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from huggingface_hub import HfApi, HfFolder, Repository, create_repo
from getpass import getpass
import os

repo = "cletus/relavance_classifier"
username = "ctngweru"
password = getpass("Enter your Hugging Face password or token: ")
create_repo(repo, exist_ok=True, token=password)
api = HfApi()
api.set_access_token(password)
token = HfFolder.get_token()
repo_url = api.repo_create(repo, token=token, exist_ok=True).git_url
local_dir = "huggingface_model"
if not os.path.exists(local_dir):
    os.makedirs(local_dir)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
tokenizer.save_pretrained(local_dir)
model.save_pretrained(local_dir)
tokenizer = DistilBertTokenizer.from_pretrained(model_save_path)
model = DistilBertForSequenceClassification.from_pretrained(model_save_path).to(device)
tokenizer.save_pretrained(local_dir)
model.save_pretrained(local_dir)
repo = Repository(local_dir, clone_from=repo_url, use_auth_token=token)
repo.push_to_hub(commit_message="Initial commit", blocking=True, use_auth_token=token)





HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-68da7042-099bd50231c49cb81f139bd3;d8839159-d7a0-4f94-82d0-7ca11f67ec54)

Invalid credentials in Authorization header

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository, create_repo

repo_id = "LMForge/text_relevance_classfier"

# Create repo if it doesn’t exist
create_repo(repo_id, exist_ok=True)

# Push your saved model folder
from huggingface_hub import upload_folder
upload_folder(
    repo_id=repo_id,
    folder_path="classifier_model",  # your saved folder
    commit_message="Initial model upload"
)

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LMForge/text_relevance_classfier/commit/10089caac408e12e1035609862443935813f5272', commit_message='Initial model upload', commit_description='', oid='10089caac408e12e1035609862443935813f5272', pr_url=None, repo_url=RepoUrl('https://huggingface.co/LMForge/text_relevance_classfier', endpoint='https://huggingface.co', repo_type='model', repo_id='LMForge/text_relevance_classfier'), pr_revision=None, pr_num=None)